# Exploring Conversation Chains and Memory with LCEL

## Install OpenAI, and LangChain dependencies

In [1]:
# Updated package versions and import paths 
# Ensures consistency with changes in earlier modules
!pip install langchain==0.3.21
!pip install langchain-openai==0.3.9
!pip install langchain-community==0.3.19
!pip install langchain-chroma==0.1.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.0 MB/s eta 0:00:00

## Enter Open AI API Key

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Setup Environment Variables

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## Load Connection to LLM

Here we create a connection to ChatGPT to use later in our chains

In [ ]:
from langchain_openai import ChatOpenAI
# Updated parameter name from model_name to model:
chatgpt = ChatOpenAI(model='gpt-4o-mini', temperature=0)

## Working with LangChain Chains

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components. Also running on multiple data points can be done easily with chains.

Chain's are the legacy interface for "chained" applications. We define a Chain very generically as a sequence of calls to components, which can include other chains.

Here we will be using LCEL chains exclusively

### The Problem with Simple LLM Chains

Simple LLM Chains cannot keep a track of past conversation history

In [ ]:
# Updated import paths for prompt templates:
from langchain.prompts import ChatPromptTemplate

prompt_txt = """{query}"""
prompt = ChatPromptTemplate.from_template(prompt_txt)

# you can also write this as llm_chain = prompt | chatgpt
llm_chain = (
    prompt
      |
    chatgpt
)

In [ ]:
response = llm_chain.invoke({'query': 'What are the first four colors of a rainbow?'})
print(response.content)

The first four colors of a rainbow are red, orange, yellow, and green.


In [ ]:
response = llm_chain.invoke({'query': 'and the other 3?'})
print(response.content)

It seems like your question might be missing some context. Could you please provide more details or clarify what you are referring to with "the other 3"? This will help me assist you better!


### Conversation Chains with LCEL

LangChain Expression Language (LCEL) connects prompts, models, parsers and retrieval components using a `|` pipe operator.

A conversation chain basically consists of user prompts, historical conversation memory and the LLM. The LLM uses the history memory to give more contextual answers to every new prompt or user query.

Memory is very important for having a true conversation with LLMs. LangChain allows us to manage conversation memory using various constructs. The main ones we will cover include:

- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationSummaryMemory
- VectorStoreRetrieverMemory
- ChatMessageHistory
- SQLChatMessageHistory

### Conversation Chains with ConversationBufferMemory

This is the simplest version of in-memory storage of historical conversation messages. It is basically a buffer for storing conversation memory.

Remember if you have a really long conversation, you might exceed the max token limit of the context window allowed for the LLM.

In [ ]:
# Updated import paths for prompt templates - using simplified paths:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

/tmp/ipykernel_633820/3875846320.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [ ]:
# function to get historical conversation messages from the memory
memory.load_memory_variables({})

{'history': []}

In [ ]:
# lets create a function now which returns the list of messages from memory
def get_memory_messages(query):
    return memory.load_memory_variables(query)['history']

get_memory_messages('What are the first four colors of a rainbow?')

[]

In [ ]:
# testing out the function with a runnable lambda which will go into our chain
# this returns the history but we also need to send our current query to the prompt
RunnableLambda(get_memory_messages).invoke({'query': 'What are the first four colors of a rainbow?'})

[]

In [ ]:
# we use a runnable passthrough to pass our current query untouched
# along with the history messages to the next step in the chain
RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ).invoke({'query': 'What are the first four colors of a rainbow?'})

{'query': 'What are the first four colors of a rainbow?', 'history': []}

In [ ]:
# creating our conversation chain now
def get_memory_messages(query):
    return memory.load_memory_variables(query)['history']

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages to next step
      |
    prompt # creates prompt using the previous two variables
      |
    chatgpt # generates response using the prompt from previous step
)

In [ ]:
query = {'query': 'What are the first four colors of a rainbow?'}
response = conversation_chain.invoke(query)
response

AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 30, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-4fe5a8e0-f57b-4710-a45b-11bf6e5fb577-0', usage_metadata={'input_tokens': 30, 'output_tokens': 17, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
print(response.content)

The first four colors of a rainbow are red, orange, yellow, and green.


In [ ]:
memory.load_memory_variables({})

{'history': []}

In [ ]:
memory.save_context(query, {"output": response.content})

In [ ]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={})]}

In [ ]:
query = {'query': 'and the other 3?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The other three colors are blue, indigo, and violet.


In [ ]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='and the other 3?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The other three colors are blue, indigo, and violet.', additional_kwargs={}, response_metadata={})]}

In [ ]:
query = {'query': 'Explain AI in 2 bullet points'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.

- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles, customer service (chatbots), and many more, enhancing efficiency and decision-making.


In [ ]:
query = {'query': 'Now do the same for Deep Learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to model complex patterns in large datasets, enabling machines to learn from data in a way that mimics human brain function.

- **Applications**: Deep Learning is widely used in image and speech recognition, natural language processing, autonomous systems, and recommendation systems, driving advancements in technologies like virtual assistants, self-driving cars, and medical imaging analysis.


In [ ]:
query = {'query': 'What have we discussed so far?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

So far, we have discussed the following topics:

1. The first four colors of a rainbow (red, orange, yellow, green) and the remaining three (blue, indigo, violet).
2. A brief explanation of Artificial Intelligence (AI) in two bullet points.
3. A brief explanation of Deep Learning in two bullet points.


In [ ]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='and the other 3?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The other three colors are blue, indigo, and violet.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Explain AI in 2 bullet points', additional_kwargs={}, response_metadata={}),
  AIMessage(content='- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.\n\n- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendat

### Conversation Chains with ConversationBufferWindowMemory

If you have a really long conversation, you might exceed the max token limit of the context window allowed for the LLM when using `ConversationBufferMemory` so `ConversationBufferWindowMemory` helps in just storing the last K conversations (one conversation piece is one user message and the corresponding AI message from the LLM) and thus helps you manage token limits and costs

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}"),
    ]
)

# stores last 2 sets of human-AI conversations
memory = ConversationBufferWindowMemory(return_messages=True, k=2)

# creating our conversation chain now
def get_memory_messages(query):
    return memory.load_memory_variables(query)['history']

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages to next step
      |
    prompt # creates prompt using the previous two variables
      |
    chatgpt # generates response using the prompt from previous step
)

/tmp/ipykernel_633820/96416143.py:15: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(return_messages=True, k=2)


In [ ]:
query = {'query': 'What are the first four colors of a rainbow?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The first four colors of a rainbow are red, orange, yellow, and green.


In [ ]:
query = {'query': 'and the other 3?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The other three colors of a rainbow are blue, indigo, and violet.


In [ ]:
query = {'query': 'Explain AI in 2 bullet points'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.

- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles, customer service (chatbots), and many more, enhancing efficiency and decision-making.


In [ ]:
query = {'query': 'Now do the same for Deep Learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to model complex patterns in large datasets, enabling machines to learn from data in a way that mimics human brain function.

- **Applications**: Deep Learning is widely used in image and speech recognition, natural language processing, autonomous systems, and generative models, driving advancements in technologies like virtual assistants, self-driving cars, and medical imaging analysis.


In [ ]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Explain AI in 2 bullet points', additional_kwargs={}, response_metadata={}),
  AIMessage(content='- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.\n\n- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles, customer service (chatbots), and many more, enhancing efficiency and decision-making.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Now do the same for Deep Learning', additional_kwargs={}, response_metadata={}),
  AIMessage(content='- **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layers (deep neural networ

In [ ]:
query = {'query': 'What have we discussed so far?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

So far, we have discussed:

1. A brief explanation of Artificial Intelligence (AI) in two bullet points, covering its definition and applications.
2. A brief explanation of Deep Learning in two bullet points, outlining its definition and applications.


In [ ]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Now do the same for Deep Learning', additional_kwargs={}, response_metadata={}),
  AIMessage(content='- **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to model complex patterns in large datasets, enabling machines to learn from data in a way that mimics human brain function.\n\n- **Applications**: Deep Learning is widely used in image and speech recognition, natural language processing, autonomous systems, and generative models, driving advancements in technologies like virtual assistants, self-driving cars, and medical imaging analysis.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What have we discussed so far?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='So far, we have discussed:\n\n1. A brief explanation of Artificial Intelligence (AI) in two bullet points, covering its definition and applications.\n2. A brief explanati

### Conversation Chains with ConversationSummaryMemory

If you have a really long conversation or a lot of messages, you might exceed the max token limit of the context window allowed for the LLM when using `ConversationBufferMemory`

`ConversationSummaryMemory` creates a summary of the conversation history over time. This can be useful for condensing information from the conversation messages over time.

This memory is most useful for longer conversations, where keeping the past message history in the prompt verbatim would take up too many tokens.

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}"),
    ]
)

memory = ConversationSummaryMemory(return_messages=True, llm=chatgpt)
# creating our conversation chain now
def get_memory_messages(query):
    return memory.load_memory_variables(query)['history']

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages as a summary to next step
      |
    prompt # creates prompt using the previous two variables
      |
    chatgpt # generates response using the prompt from previous step
)

/tmp/ipykernel_633820/2640930987.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(return_messages=True, llm=chatgpt)


In [ ]:
query = {'query': 'Explain AI in 2 bullet points'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.

- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles, customer service (chatbots), and more, enhancing efficiency and decision-making.


In [ ]:
query = {'query': 'Now do the same for Deep Learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- Deep Learning is a subset of machine learning that uses neural networks with many layers to analyze and learn from large amounts of data, enabling complex pattern recognition.
- It is widely used in applications such as image and speech recognition, natural language processing, and autonomous systems, significantly improving accuracy and performance in these areas.


In [ ]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human asks the AI to explain artificial intelligence in two bullet points. The AI defines artificial intelligence as the simulation of human intelligence in machines that can think and learn, listing applications in various fields such as healthcare, finance, autonomous vehicles, and customer service, highlighting its role in enhancing efficiency and decision-making. The human then requests a similar explanation for Deep Learning, and the AI describes it as a subset of machine learning that uses neural networks with many layers for complex pattern recognition and notes its applications in image and speech recognition, natural language processing, and autonomous systems, which significantly improve accuracy and performance.', additional_kwargs={}, response_metadata={})]}

In [ ]:
query = {'query': 'What have we discussed so far?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

We discussed artificial intelligence, defining it as the simulation of human intelligence in machines, with applications in various fields like healthcare and finance. We also covered deep learning, describing it as a subset of machine learning that uses neural networks for complex pattern recognition, with applications in image and speech recognition, among others.


### Conversation Chains with VectorStoreRetrieverMemory

`VectorStoreRetrieverMemory` stores historical conversation messages in a vector store and queries the top-K most "relevant" history messages every time it is called.

This differs from most of the other Memory classes in that it doesn't explicitly track the order of interactions but retrieves history based on embedding similarity to the current question or prompt.

In this case, the "docs" are previous conversation snippets. This can be useful to refer to relevant pieces of information that the AI was told earlier in the conversation.

#### Connect to  Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [ ]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

#### Create a Vector Database to store conversation history

Here we use the Chroma vector DB and initialize an empty database collection to store conversation messages

In [ ]:
from langchain_chroma import Chroma

# create empty vector DB
chroma_db = Chroma(collection_name='history_db',
                   embedding_function=openai_embed_model)

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import VectorStoreRetrieverMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}"),
    ]
)

# load 2 most similar conversation messages from vector db history for each new message \ prompt
# this uses cosine embedding similarity to load the top 2 similar messgages to the input prompt \ query
retriever = chroma_db.as_retriever(search_type="similarity",
                                   search_kwargs={"k": 2})
memory = VectorStoreRetrieverMemory(retriever=retriever, return_messages=True)

# creating our conversation chain now
def get_memory_messages(query):
    return [memory.load_memory_variables(query)['history']]

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages to next step
      |
    prompt # creates prompt using the previous two variables
      |
    chatgpt # generates response using the prompt from previous step
)

/tmp/ipykernel_633820/573889585.py:18: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = VectorStoreRetrieverMemory(retriever=retriever, return_messages=True)


In [ ]:
query = {'query': 'Tell me about AI'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn. It encompasses various technologies, including machine learning, natural language processing, and robotics. AI can perform tasks such as speech recognition, image analysis, and decision-making, and is used in applications ranging from virtual assistants to autonomous vehicles.


In [ ]:
query = {'query': 'What about deep learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


Deep learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to analyze and learn from large amounts of data. It excels in tasks such as image and speech recognition, natural language processing, and game playing. Deep learning models automatically extract features from raw data, making them highly effective for complex problems.


In [ ]:
query = {'query': 'Tell me about the fastest animal in the world in 2 lines'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The fastest animal in the world is the peregrine falcon, which can reach speeds over 240 mph (386 km/h) during its hunting stoop (high-speed dive). On land, the cheetah holds the record, capable of sprinting up to 60-70 mph (97-113 km/h) in short bursts.


In [ ]:
query = {'query': 'What about the cheetah?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The cheetah is the fastest land animal, capable of reaching speeds of 60-70 mph (97-113 km/h) in short bursts covering distances up to 1,500 feet (460 meters). Its unique adaptations, such as a lightweight body, long legs, and large nasal passages, enable it to accelerate rapidly and chase down prey.


Now for a new query around machine learning even if the most recent conversation messages have been about animals, it uses the vector databases to load the last 2 historical conversations which are closest to the current question in terms of semantic similarity

In [ ]:
print(memory.load_memory_variables({'query': 'What about machine learning?'})['history'])

query: What about deep learning
output: Deep learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to analyze and learn from large amounts of data. It excels in tasks such as image and speech recognition, natural language processing, and game playing. Deep learning models automatically extract features from raw data, making them highly effective for complex problems.
query: Tell me about AI
output: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn. It encompasses various technologies, including machine learning, natural language processing, and robotics. AI can perform tasks such as speech recognition, image analysis, and decision-making, and is used in applications ranging from virtual assistants to autonomous vehicles.


In [ ]:
query = {'query': 'What about machine learning?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Machine learning is a subset of artificial intelligence that focuses on the development of algorithms that allow computers to learn from and make predictions or decisions based on data. It involves training models on datasets to identify patterns and improve performance over time without being explicitly programmed for specific tasks. Machine learning is widely used in applications like recommendation systems, fraud detection, and predictive analytics.


### Multi-user Conversation Chains with ChatMessageHistory

The concept of `ChatHistory` refers to a class in LangChain which can be used to wrap an arbitrary chain. This `ChatHistory` will keep track of inputs and outputs of the underlying chain, and append them as messages to a message database. Future interactions will then load those messages and pass them into the chain as part of the input.

The beauty of `ChatMessageHistory` is that we can store separate conversation histories per user or session which is often the need for real-world chatbots which will be accessed by many users at the same time.

We use a `get_session_history` function which is expected to take in a `session_id` and return a Message History object. Everything is stored in memory. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain

In [ ]:
from langchain.chat_message_histories import ChatMessageHistory
from langchain.chat_history import BaseChatMessageHistory
from langchain.runnables.history import RunnableWithMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
# used to retrieve conversation history from memory
# based on a specific user or session ID
history_store = {}
def get_session_history(session_id):
    if session_id not in history_store:
        history_store[session_id] = ChatMessageHistory()
    return history_store[session_id]

# prompt to load in history and current input from the user
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{human_input}"),
    ]
)

# create a basic LLM Chain
llm_chain = (prompt_template
                |
             chatgpt)

# create a conversation chain which can load memory based on specific user or session id
conv_chain = RunnableWithMessageHistory(
    llm_chain,
    get_session_history,
    input_messages_key="human_input",
    history_messages_key="history",
)

# create a utility function to take in current user input prompt and their session ID
# streams result live back to the user from the LLM
def chat_with_llm(prompt: str, session_id: str):
    for chunk in conv_chain.stream({"human_input": prompt},
                                   {'configurable': { 'session_id': session_id}}):
        print(chunk.content, end="")

Test conversation chain for user 1

In [ ]:
user_id = 'bob123'
prompt = "Hi I am Bob, can you explain AI in 3 bullet points?"
chat_with_llm(prompt, user_id)

Sure, Bob! Here are three key points about AI:

1. **Definition and Purpose**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn. Its primary purpose is to perform tasks that typically require human intelligence, such as problem-solving, understanding natural language, and recognizing patterns.

2. **Types of AI**: AI can be categorized into two main types: Narrow AI, which is designed for specific tasks (like virtual assistants or recommendation systems), and General AI, which aims to understand and reason across a wide range of tasks, similar to human cognitive abilities (though this is still largely theoretical).

3. **Applications**: AI is used in various fields, including healthcare (for diagnostics and personalized medicine), finance (for fraud detection and algorithmic trading), transportation (in autonomous vehicles), and customer service (through chatbots and virtual agents), among many others.

In [ ]:
prompt = "Now do the same for deep learning"
chat_with_llm(prompt, user_id)

Of course, Bob! Here are three key points about deep learning:

1. **Definition and Structure**: Deep learning is a subset of machine learning that uses neural networks with many layers (hence "deep") to model complex patterns in large datasets. These neural networks are inspired by the structure and function of the human brain.

2. **Learning Process**: Deep learning algorithms learn from vast amounts of data through a process called training, where they adjust their internal parameters to minimize the difference between their predictions and the actual outcomes. This allows them to automatically extract features and improve their performance over time.

3. **Applications**: Deep learning has led to significant advancements in various fields, including image and speech recognition, natural language processing, and game playing (like AlphaGo). It powers technologies such as facial recognition systems, language translation services, and self-driving cars.

In [ ]:
prompt = "Discuss briefly what have we discussed so far is bullet points?"
chat_with_llm(prompt, user_id)

Certainly! Here’s a brief summary of what we’ve discussed so far in bullet points:

### Artificial Intelligence (AI)
- **Definition and Purpose**: AI simulates human intelligence in machines to perform tasks that require cognitive functions.
- **Types of AI**: Includes Narrow AI (specific tasks) and General AI (broad cognitive abilities, mostly theoretical).
- **Applications**: Used in healthcare, finance, transportation, and customer service, among other fields.

### Deep Learning
- **Definition and Structure**: A subset of machine learning using deep neural networks to model complex patterns in data.
- **Learning Process**: Involves training on large datasets to automatically extract features and improve predictions.
- **Applications**: Powers advancements in image/speech recognition, natural language processing, and technologies like facial recognition and self-driving cars.

Now test conversation chain for user 2

In [ ]:
user_id = 'james007'
prompt = "Hi can you explain what is an LLM in 2 bullet points?"
chat_with_llm(prompt, user_id)

Sure! Here are two key points about LLMs (Large Language Models):

- **Definition**: LLMs are advanced artificial intelligence models designed to understand, generate, and manipulate human language by processing vast amounts of text data, enabling them to perform tasks like translation, summarization, and conversation.

- **Architecture**: They typically use deep learning techniques, particularly transformer architectures, which allow them to capture complex patterns and relationships in language, making them highly effective for various natural language processing applications.

In [ ]:
prompt = "Actually I meant in the context of AI?"
chat_with_llm(prompt, user_id)

Got it! Here’s a refined explanation of LLMs in the context of AI:

- **Large Language Models (LLMs)**: These are AI models that are trained on extensive datasets to understand and generate human-like text. They leverage deep learning techniques to predict the next word in a sentence, enabling them to perform a wide range of language-related tasks.

- **Applications**: LLMs are used in various applications, including chatbots, content generation, language translation, and sentiment analysis, making them valuable tools in fields like customer service, education, and creative writing.

In [ ]:
prompt = "Summarize briefly what we have discussed so far?"
chat_with_llm(prompt, user_id)

We discussed Large Language Models (LLMs) in the context of AI, highlighting two main points: 

1. LLMs are advanced AI models trained on large datasets to understand and generate human-like text, utilizing deep learning techniques.
2. They have diverse applications, including chatbots, content generation, language translation, and sentiment analysis, making them useful in various fields.

In [ ]:
user_id = 'bob123'
prompt = "Discuss briefly what have we discussed so far is bullet points?"
chat_with_llm(prompt, user_id)

Sure! Here’s a concise summary of our discussion in bullet points:

### Summary of Discussion
- **Artificial Intelligence (AI)**:
  - Simulates human intelligence in machines for cognitive tasks.
  - Includes Narrow AI (specific tasks) and General AI (theoretical broad abilities).
  - Applications span healthcare, finance, transportation, and customer service.

- **Deep Learning**:
  - A subset of machine learning using deep neural networks to analyze complex data.
  - Involves training on large datasets to enhance feature extraction and predictions.
  - Applications include image/speech recognition, natural language processing, and technologies like facial recognition and autonomous vehicles.

### Multi-user Window-based Conversation Chains with persistence - SQLChatMessageHistory

The beauty of `SQLChatMessageHistory` is that we can store separate conversation histories per user or session which is often the need for real-world chatbots which will be accessed by many users at the same time. Instead of in-memory we can store it in a SQL database which can be used to store a lot of conversations.

We use a `get_session_history` function which is expected to take in a `session_id` and return a Message History object. Everything is stored in a SQL database. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain

We also use a `memory_buffer_window` function to only use the top-K last historical conversations before sending it to the LLM, basically our own implementation of `ConversationBufferWindowMemory`

In [ ]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove all conversation histories
!rm memory.db

In [ ]:
from langchain.chat_message_histories import SQLChatMessageHistory
from langchain.chat_history import BaseChatMessageHistory
from langchain.runnables.history import RunnableWithMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.runnables import RunnablePassthrough

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

# prompt to load in history and current input from the user
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{human_input}"),
    ]
)

# create a memory buffer window function to return the last K conversations
def memory_buffer_window(messages, k=2):
    return messages[-(k+1):]

# create a basic LLM Chain which only sends the last K conversations per user
llm_chain = (
    RunnablePassthrough.assign(history=lambda x: memory_buffer_window(x["history"]))
      |
    prompt_template
      |
    chatgpt
)

In [ ]:
# create a conversation chain which can load memory based on specific user or session id
conv_chain = RunnableWithMessageHistory(
    llm_chain,
    get_session_history_db,
    input_messages_key="human_input",
    history_messages_key="history",
)

# create a utility function to take in current user input prompt and their session ID
# streams result live back to the user from the LLM
def chat_with_llm(prompt: str, session_id: str):
    for chunk in conv_chain.stream({"human_input": prompt},
                                   {'configurable': { 'session_id': session_id}}):
        print(chunk.content, end="")

Test conversation chain for user 1

In [ ]:
user_id = 'jim001'
prompt = "Hi can you tell me which is the fastest animal?"
chat_with_llm(prompt, user_id)

/home/santhosh/anaconda3/lib/python3.11/site-packages/langchain_core/runnables/history.py:608: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


The fastest animal in the world is the peregrine falcon. When in a dive, it can reach speeds of over 240 miles per hour (386 kilometers per hour). If you're considering speed in level flight, the Brazilian free-tailed bat holds the record, flying at speeds of around 99 miles per hour (160 kilometers per hour). For land animals, the cheetah is the fastest, capable of running speeds up to 60-70 miles per hour (97-113 kilometers per hour) in short bursts.

In [ ]:
prompt = "what about the slowest animal?"
chat_with_llm(prompt, user_id)

The slowest animal is often considered to be the three-toed sloth. It moves at an average speed of about 0.24 kilometers per hour (0.15 miles per hour) when climbing trees. In water, they can swim slightly faster, but they are still quite slow compared to many other animals. Another contender for the title of slowest animal is the garden snail, which moves at a speed of about 0.03 miles per hour (0.048 kilometers per hour). Both of these animals have adapted to their slow pace as part of their survival strategies.

In [ ]:
prompt = "what about the largest animal?"
chat_with_llm(prompt, user_id)

The largest animal on Earth is the blue whale (*Balaenoptera musculus*). Blue whales can reach lengths of up to 100 feet (30 meters) or more and can weigh as much as 200 tons (approximately 181 metric tonnes). These magnificent marine mammals are not only the largest animals alive today but are also believed to be the largest animals to have ever existed on Earth, surpassing even the largest dinosaurs. Their immense size is supported by their diet, which primarily consists of tiny shrimp-like animals called krill.

In [ ]:
prompt = "what topics have we discussed, show briefly as bullet points"
chat_with_llm(prompt, user_id)

Sure! Here are the topics we've discussed so far:

- The slowest animal (three-toed sloth and garden snail)
- The largest animal (blue whale)

Now test conversation chain for user 2

In [ ]:
user_id = 'john005'
prompt = "Explain AI in 3 bullets to a child"
chat_with_llm(prompt, user_id)

Sure! Here are three simple points to explain AI to a child:

1. **Smart Helpers**: AI is like a smart robot or computer that can help us with tasks, like answering questions or playing games, by learning from lots of information.

2. **Learning from Experience**: Just like how you get better at something the more you practice, AI learns from examples and gets better at doing things over time.

3. **Talking and Understanding**: AI can understand what we say and even talk back to us, kind of like having a conversation with a friend who knows a lot of things!

In [ ]:
prompt = "Now do the same for Generative AI"
chat_with_llm(prompt, user_id)

Of course! Here are three simple points to explain Generative AI to a child:

1. **Creative Robot**: Generative AI is like a creative robot that can make new things, like pictures, stories, or music, all by itself!

2. **Learning from Examples**: It learns by looking at lots of examples, like reading many books or seeing many drawings, so it can create something new that’s similar but different.

3. **Imagination in a Computer**: Just like you use your imagination to come up with fun ideas, Generative AI uses its "imagination" to invent cool stuff that we can enjoy!

In [ ]:
prompt = "Now do the same for machine learning"
chat_with_llm(prompt, user_id)

Sure! Here are three simple points to explain machine learning to a child:

1. **Learning Like You**: Machine learning is when a computer learns from experience, just like you learn new things by practicing and trying again.

2. **Finding Patterns**: It looks at lots of information to find patterns, like how you notice that the sky is blue during the day or that ice cream is cold and sweet.

3. **Getting Better Over Time**: The more it practices, the better it gets at making decisions or predictions, like guessing what your favorite game is based on what you’ve played before!

In [ ]:
prompt = "what topics have we discussed, show briefly as bullet points"
chat_with_llm(prompt, user_id)

Sure! Here are the topics we've discussed so far:

- **Generative AI**
  - Creative robot that makes new things
  - Learns from examples
  - Uses imagination to invent cool stuff

- **Machine Learning**
  - Learning from experience like a person
  - Finding patterns in information
  - Improving over time with practice